In [1]:
import cv2 # opencv 사용

import numpy as np
# =========================== 

def mark_img(img, blue_threshold=200, green_threshold=200, red_threshold=200): # 1. 흰색 차선 찾기
    mark = img
    #  BGR 제한 값 (기준)
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]

    # BGR 제한 값(기준)보다 작으면 검은색으로
    thresholds = (img[:,:,0] < bgr_threshold[0]) \
                | (img[:,:,1] < bgr_threshold[1]) \
                | (img[:,:,2] < bgr_threshold[2])
    mark[thresholds] = [0,0,0] # 검정색으로 
    return mark
 
def grayscale(img): # 흑백이미지로 변환
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def region_of_interest(img, vertices, color3=(255,255,255), color1=255): # ROI 셋팅
    mask = np.zeros_like(img) # mask = img와 같은 크기의 빈 이미지
    
    if len(img.shape) > 2: # Color 이미지(3채널)라면 :
        color = color3
    else: # 흑백 이미지(1채널)라면 :
        color = color1
        
    # vertices에 정한 점들로 이뤄진 다각형부분(ROI 설정부분)을 color로 채움 
    cv2.fillPoly(mask, vertices, color)
    # 이미지와 color로 채워진 ROI를 합침
    ROI_image = cv2.bitwise_and(img, mask)

    return ROI_image

# 허프변환 라인 그리기 (확인용)
def draw_lines(img, lines, color=[255, 255, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)\
            
# 허프 변환
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap): 
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)

    return lines

# 두 이미지 operlap 하기
def weighted_img(img, initial_img, α=1, β=1., λ=0.): 
    return cv2.addWeighted(initial_img, α, img, β, λ)

# '대표선' 구하기 (62, 63줄 코드 이해 X )
def get_fitline(img, f_lines): 
    lines = np.squeeze(f_lines)
    if(len(lines.shape) == 1): #선이 하나밖에 없는 경우 배열의 모양 따로 조정
        lines = lines.reshape(2,2)
    else:
        lines = lines.reshape(lines.shape[0]*2,2)

    rows,cols = img.shape[:2]
    output = cv2.fitLine(lines,cv2.DIST_L2,0, 0.01, 0.01)
    vx, vy, x, y = output[0], output[1], output[2], output[3]
    x1, y1 = int(((img.shape[0]/2+70)-y)/vy*vx + x) , int(img.shape[0]/2+70)
    x2, y2 = int(((img.shape[0]/2-25)-y)/vy*vx + x) , int(img.shape[0]/2-25)
    result = [x1,y1,x2,y2]
    
    return result

# '대표선' 그리기
def draw_fit_line(img, lines, color=[255, 0, 0], thickness=10):
        # cv2.line(이미지, 시작좌표(0, 0), 끝좌표(500, 500), 색깔, 두께)
        cv2.line(img, (lines[0], lines[1]), (lines[2], lines[3]), color, thickness) 

# '대표선' 이용해서 -> 소실점 구하기 (공식)
def expression(x1,y1,x2,y2,x3,y3,x4,y4):
    m_a = (y2 - y1) / (x2 -x1)
    m_b = (y4 - y3) / (x4 - x3)
    n_a = -((y2 - y1) / (x2 - x1) * x1 ) + y1
    n_b = -((y4 - y3) / (x4 -x3) * x3 ) + y3
    x = (n_b - n_a) / (m_a - m_b) 
    y = m_a * ((n_b - n_a) / (m_a - m_b)) + n_a 

    return x,y

#cap = cv2.VideoCapture(0) # 960x540
cap = cv2.VideoCapture('테스트영상4.mp4') # 1920x1080 -> 에러
 
while(cap.isOpened()):
    ret,image = cap.read()
    #image = cv2.resize(image, (640, 360)) # window 사이즈 조정
    
    height, width = image.shape[:2] # 이미지 높이, 너비
    m_width = int(width/2)

    mrk_img = mark_img(image)
    gray_img = grayscale(mrk_img)
    ####################################################################
    vertices = np.array([[(350,height-150),
                          (600,360),
                          (800,360),
                          (width-100,height-150)]], dtype=np.int32)
    ####################################################################
    ROI_img = region_of_interest(gray_img, vertices) # ROI 설정
    ####################################################################
    rho = 1
    theta = 1 * np.pi/180
    threshold = 30    # threshold 값이  작으면 그만큼 기준이 낮아져 많은 직선이 검출될 것이고, 값을 높게 정하면 그만큼 적지만 확실한 직선들만 검출이 될 것이다
    ####################################################################
    
    line_arr = hough_lines(ROI_img, rho, theta, threshold, 10, 20) # 허프 변환
    line_arr = np.squeeze(line_arr) # remove single dimension (차원을 하나 줄임)

    # 기울기 구하기 (arctan(y,x)이용)
    slope_degree = np.arctan2(line_arr[:,1] - line_arr[:,3], line_arr[:,0] - line_arr[:,2]) * 180 / np.pi

# 수평 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)<175]
    slope_degree = slope_degree[np.abs(slope_degree)<175]
# 수직 기울기 제한
    line_arr = line_arr[np.abs(slope_degree)>95]
    slope_degree = slope_degree[np.abs(slope_degree)>95]
# 필터링된 직선 버리기
    L_lines, R_lines = line_arr[(slope_degree>0),:], line_arr[(slope_degree<0),:]
    L_lines, R_lines = L_lines[:,None], R_lines[:,None]

    if(len(L_lines) == 0 and len(R_lines) == 0): #L_lines, R_lines 모두 없는 경우
        L_lines = pre_left_line
        R_lines = pre_right_line
    elif(len(L_lines) == 0):#L_lines만 없는 경우
        L_lines = pre_left_line
        pre_right_line = R_lines
    elif(len(R_lines) == 0):#R_lines만 없는 경우
        R_lines = pre_right_line
        pre_left_line = L_lines
    else:#라인 모두 검출한 경우
        pre_right_line = R_lines
        pre_left_line = L_lines

    temp = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

# 왼쪽, 오른쪽 각각 대표선 구하기
    left_fit_line = get_fitline(image,L_lines)
    right_fit_line = get_fitline(image,R_lines)
    #print(left_fit_line) # [158, 539, 388, 370] -> 단 1개 검출 

# 대표선 '그리기'
    draw_fit_line(temp, left_fit_line)
    draw_fit_line(temp, right_fit_line)
    #print('left_fit_line = ', left_fit_line) # [158, 539, 388, 370]

    vanishing_point = expression(left_fit_line[0],left_fit_line[1],left_fit_line[2],left_fit_line[3],right_fit_line[0],right_fit_line[1],right_fit_line[2],right_fit_line[3])
    #print(vanishing_point) # (476.9880952380953, 304.61309523809524)...

    v_x = int(vanishing_point[0])
    v_y = int(vanishing_point[1])

    result = weighted_img(temp, image) # 원본 이미지(=image)에 검출된 선(=temp) overlap
    cv2.circle(result, (v_x,v_y), 6, (0,0,255), -1) # cv2.circle(image, center_coordinates, radius, color, thickness)

    #circle 기준선(보조선)
    cv2.line(result,(m_width,0),(m_width,300),(255,255,0),5) # cv2.line(image, start_point, end_point, color, thickness)
    
    if(m_width != v_x):
        slope_degree = float(v_y) / (m_width - v_x)
    else:
        slope_degree = 0
    print(slope_degree)
    if(slope_degree > 2.475): # degree 22
        print("Right!!!")
        #cv2.circle(result,(1000,50), 6,(0,0,255),-1) # (1000,50)에 circle 찍어라 
    elif(slope_degree < -2.475): # 소실점의 x좌표가 중앙선보다 왼쪽에 있을때
        print("Left!!!")
        #cv2.circle(result,(100,50), 6,(0,0,255),-1) # (100,50)에 circle 찍어라
    else:
        print("foward!!!")
    
    cv2.imshow('result2',result) # 결과 이미지 출력
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()
    


-3.212121212121212
Left!!!
-4.311688311688312
Left!!!
-4.955882352941177
Left!!!
-4.589041095890411
Left!!!
-4.2025316455696204
Left!!!
-4.2025316455696204
Left!!!
-4.189873417721519
Left!!!
-4.189873417721519
Left!!!
-4.426666666666667
Left!!!
-4.652777777777778
Left!!!
-4.869565217391305
Left!!!
-4.785714285714286
Left!!!
-4.426666666666667
Left!!!
-4.690140845070423
Left!!!
-4.869565217391305
Left!!!
-4.302631578947368
Left!!!
-4.811594202898551
Left!!!
-5.328125
Left!!!
-5.3125
Left!!!
-5.296875
Left!!!
-5.5
Left!!!
-5.557377049180328
Left!!!
-5.3125
Left!!!
-5.296875
Left!!!
-5.349206349206349
Left!!!
-5.333333333333333
Left!!!
-5.265625
Left!!!
-4.583333333333333
Left!!!
-4.493150684931507
Left!!!
-4.7
Left!!!
-4.445945945945946
Left!!!
-5.728813559322034
Left!!!
-5.138461538461539
Left!!!
-5.540983606557377
Left!!!
-4.897058823529412
Left!!!
-4.826086956521739
Left!!!
-4.259740259740259
Left!!!
-4.246753246753247
Left!!!
-5.65
Left!!!
-4.373333333333333
Left!!!
-4.75714285714285